# Imports

In [18]:
import pandas as pd
import numpy as np
import random
import re

In [19]:
data_DB = pd.read_csv("Food_DB.csv")

In [20]:
# Subset of list
data_dinner = data_DB[pd.isnull(data_DB["Lunch"])]
data_lunch = data_DB[data_DB["Lunch"] == "x"]

In [21]:
data_dinner.head()

,Meal,Amount,Stars (1-5),Effort (1-5),Lunch,Ingredients,Notes
0,Bacon ärt pasta,5,5,4,NaN,"[Bacon=280, Spaghetti=500, Ärter=80, Matgrädde...",NaN
1,Burritobowl,5,4,4,NaN,"[Maletkött=400, Vitlök=0.2, Kidneybönor=230, P...",NaN
2,Butterchicken,5,2,4,NaN,NaN,NaN
3,Curry höna,4,3,3,NaN,"[HönsStrimlorMarinad=500, Matgrädde=2, Paprika...",NaN
4,Curry sill,4,4,4,NaN,NaN,NaN


# Main functions

In [5]:
def pick_random_meal(meal_db, min_amount=0, max_amount=None, limit_selection=True):
    global chosen_meals
    
    if max_amount is not None:
        eligible_meals = meal_db[
            (meal_db['Amount'] >= min_amount) & 
            (meal_db['Amount'] <= max_amount) &
            (~meal_db['Meal'].isin(chosen_meals))
        ]
    else:
        eligible_meals = meal_db[
            (meal_db['Amount'] >= min_amount) &
            (~meal_db['Meal'].isin(chosen_meals))
        ]

    if not eligible_meals.empty:
        random_row = eligible_meals.sample()
        random_meal = random_row.iloc[0]

        current_meal = random_meal
        
        if limit_selection == True:
            chosen_meals.append(current_meal[0])
        
        return current_meal
    else:
        current_meal = "No meal options available."
        return current_meal

In [6]:
def generate_meal_plan(data_lunch, data_dinner, num_days, single_lunch_days, ready_lunch_days):
    meal_plan = pd.DataFrame(index=['lunch', 'dinner'])
    
    global meals_left
    global leftover_meal_1
    global leftover_amount_1
    global leftover_meal_2
    global leftover_amount_2
    
    meal_placeholder = None
    
    # Saves requirements for next lunch
    next_lunch_ready = False
    next_lunch_single = False

    for day in range(1, num_days + 1):
        
        print(f"start of day:{day}")
        print(leftover_meal_1)
        print(leftover_amount_1)
        print(leftover_meal_2)
        print(leftover_amount_2)
        
        
        
        lunch_choice, dinner_choice = None, None
        

        
        # Check next lunch for single or ready made requirement
        if day + 1 in ready_lunch_days:
            next_lunch_ready = True
        else:
            next_lunch_ready = False
        
        if day in single_lunch_days:
            next_lunch_single = True
        else:
            next_lunch_single = False
            
            
            
        # CHECK LEFTOVER STATE (move forward form 2 if 1 empty)
        if leftover_amount_1 <= 0 and leftover_amount_2 != 0:
            leftover_meal_1 = leftover_meal_2
            leftover_amount_1 = leftover_amount_2
            
            leftover_meal_2 = None
            leftover_amount_2 = 0
            
        # LUNCH SELECTION
        
        # Check if there are leftovers, otherwise choose a new lunch from lunch db
        if next_lunch_single == True:
            if leftover_amount_1 == 1:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 1

                # At this point leftovers 1 is empty, moving 2 into 1
                leftover_meal_1 = leftover_meal_2
                leftover_amount_1 = leftover_amount_2

                leftover_meal_2 = None
                leftover_amount_2 = 0

            elif leftover_amount_1 > 1:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 1
            else:
                lunch_choice = pick_random_meal(data_lunch, min_amount=0, limit_selection=False)["Meal"]
        else:
            if leftover_amount_1 == 1:
                if leftover_amount_2 >= 1:
                    lunch_choice = f"{leftover_meal_1} + {leftover_meal_2}"
                    leftover_amount_1 -= 1
                    leftover_amount_2 -= 1

                    # At this point leftovers 1 is empty, moving 2 into 1
                    leftover_meal_1 = leftover_meal_2
                    leftover_amount_1 = leftover_amount_2

                    leftover_meal_2 = None
                    leftover_amount_2 = 0
                else:
                    lunch_choice = f"{leftover_meal_1} + something"
                    leftover_amount_1 -= 1

            elif leftover_amount_1 == 2:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 2

                # At this point leftovers 1 is empty, moving 2 into 1
                leftover_meal_1 = leftover_meal_2
                leftover_amount_1 = leftover_amount_2

                leftover_meal_2 = None
                leftover_amount_2 = 0

            elif leftover_amount_1 > 2:
                lunch_choice = leftover_meal_1
                leftover_amount_1 -= 2
            else:
                lunch_choice = pick_random_meal(data_lunch, min_amount=0, limit_selection=False)["Meal"]
            
        
        # DINNER SELECTION
        
        # In case not ready and not single lunch, pick dinner
        if next_lunch_ready == False:
            if leftover_amount_1 + leftover_amount_2 >=2:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, max_amount=2, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
            
                meals_left -=2
                
            else:  
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]

                meals_left -=2
        elif next_lunch_ready == True:
            if leftover_amount_1 + leftover_amount_2 < 2:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=4, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
            
                meals_left -=2
            else:
                meal_placeholder = pick_random_meal(data_dinner, min_amount=0, limit_selection=True)
                dinner_choice = meal_placeholder["Meal"]
                meals_left = meal_placeholder["Amount"]
                
                meals_left -=2

        # Update the meal plan
        meal_plan.at['lunch', day] = lunch_choice
        meal_plan.at['dinner', day] = dinner_choice
        
        # At the end of day, check dinner leftovers and move to leftover variables
        if meals_left > 0:
            leftover_meal_2 = dinner_choice
            leftover_amount_2 = meals_left
            
            meals_left = 0
            
        


    return meal_plan

In [16]:
# Example usage:
num_days = 7
single_lunch_days = []
ready_lunch_days = [2,3,4,5,6]
chosen_meals = []
leftover_meal_1 = None
leftover_amount_1 = 0
leftover_meal_2 = None
leftover_amount_2 = 0

meal_plan_result = generate_meal_plan(data_lunch, data_dinner, num_days, single_lunch_days, ready_lunch_days)
print(meal_plan_result)

start of day:1
None
0
None
0
start of day:2
None
0
Lasagne
3
start of day:3
Lasagne
1
Feta spenat pasta
3
start of day:4
Feta spenat pasta
2
None
0
start of day:5
None
0
Curry höna
2
start of day:6
None
0
Onsdags maletköttsås
3
start of day:7
Onsdags maletköttsås
1
Gnocchi tomat
3
              1                  2                            3  \
lunch   Plättar            Lasagne  Lasagne + Feta spenat pasta   
dinner  Lasagne  Feta spenat pasta                   Fiskpinnar   

                        4                     5                     6  \
lunch   Feta spenat pasta            Curry höna  Onsdags maletköttsås   
dinner         Curry höna  Onsdags maletköttsås         Gnocchi tomat   

                                           7  
lunch   Onsdags maletköttsås + Gnocchi tomat  
dinner                         Knackkorvssås  
['Lasagne', 'Feta spenat pasta', 'Fiskpinnar', 'Curry höna', 'Onsdags maletköttsås', 'Gnocchi tomat', 'Knackkorvssås']


# Ingredient Functions

In [23]:
print(chosen_meals)

['Lasagne', 'Feta spenat pasta', 'Fiskpinnar', 'Curry höna', 'Onsdags maletköttsås', 'Gnocchi tomat', 'Knackkorvssås']


In [22]:
# Testing indregients list function

ingredients_str = data_dinner["Ingredients"][0]
print(ingredients_str)
ingredients_list = re.findall(r'(\w+)\s*=\s*(\d+)', ingredients_str)
print(ingredients_list)

[Bacon=280, Spaghetti=500, Ärter=80, Matgrädde=4, Champinjoner=400, Vitlök=0.2]
[('Bacon', '280'), ('Spaghetti', '500'), ('Ärter', '80'), ('Matgrädde', '4'), ('Champinjoner', '400'), ('Vitlök', '0')]


In [30]:
data_dinner.head()

,Meal,Amount,Stars (1-5),Effort (1-5),Lunch,Ingredients,Notes
0,Bacon ärt pasta,5,5,4,NaN,"[Bacon=280, Spaghetti=500, Ärter=80, Matgrädde...",NaN
1,Burritobowl,5,4,4,NaN,"[Maletkött=400, Vitlök=0.2, Kidneybönor=230, P...",NaN
2,Butterchicken,5,2,4,NaN,NaN,NaN
3,Curry höna,4,3,3,NaN,"[HönsStrimlorMarinad=500, Matgrädde=2, Paprika...",NaN
4,Curry sill,4,4,4,NaN,NaN,NaN


In [57]:
total_ingredients = {}

In [58]:
for meal in chosen_meals:
    print(f"Ingredients for meal {meal}:")
    
    # Get the row where the Meal column matches the current chosen meal
    row = data_dinner[data_dinner["Meal"] == meal]

    if not row.empty:
        # Get the ingredients string for the current meal
        ingredients_str = row["Ingredients"].values[0]

        # Check if the value is a valid string (not NaN)
        if pd.notna(ingredients_str):
            # Using regular expression to find key-value pairs in the string
            matches = re.findall(r'(\w+)\s*=\s*(\d+)', ingredients_str)

            # Extracting individual ingredients
            for match in matches:
                ingredient, quantity = match
                print(f"  Ingredient: {ingredient}, Quantity: {quantity}")

                # Accumulate total quantity for each ingredient
                if ingredient in total_ingredients:
                    total_ingredients[ingredient] += int(quantity)
                else:
                    total_ingredients[ingredient] = int(quantity)
        else:
            print(f"  No ingredients found for meal {meal}.")
    else:
        print(f"  Meal {meal} not found in the DataFrame.")

Ingredients for meal Lasagne:
  Ingredient: Maletkött, Quantity: 400
  Ingredient: Lök, Quantity: 2
  Ingredient: Tomatkross, Quantity: 500
  Ingredient: Smör, Quantity: 40
  Ingredient: Mjöl, Quantity: 1
  Ingredient: MjölkRöd, Quantity: 10
  Ingredient: Emmental, Quantity: 150
  Ingredient: Lasagneplattor, Quantity: 200
Ingredients for meal Feta spenat pasta:
  Ingredient: FetaOlja, Quantity: 75
  Ingredient: HönsStrimlor, Quantity: 450
  Ingredient: Spenat, Quantity: 100
  Ingredient: Vitlök, Quantity: 0
  Ingredient: Farfalle, Quantity: 500
Ingredients for meal Fiskpinnar:
  Ingredient: Potatismos, Quantity: 0
  Ingredient: Fiskpinnar, Quantity: 600
  Ingredient: RemouladeSås, Quantity: 0
Ingredients for meal Curry höna:
  Ingredient: HönsStrimlorMarinad, Quantity: 500
  Ingredient: Matgrädde, Quantity: 2
  Ingredient: Paprika, Quantity: 1
  Ingredient: Ris, Quantity: 5
Ingredients for meal Onsdags maletköttsås:
  Ingredient: Maletkött, Quantity: 400
  Ingredient: Lök, Quantity: 0


In [59]:
# Output total ingredients
print("\nTotal Ingredients:")
for ingredient, total_quantity in total_ingredients.items():
    print(f"  Ingredient: {ingredient}, Total Quantity: {total_quantity}")


Total Ingredients:
  Ingredient: Maletkött, Total Quantity: 800
  Ingredient: Lök, Total Quantity: 2
  Ingredient: Tomatkross, Total Quantity: 500
  Ingredient: Smör, Total Quantity: 40
  Ingredient: Mjöl, Total Quantity: 1
  Ingredient: MjölkRöd, Total Quantity: 10
  Ingredient: Emmental, Total Quantity: 150
  Ingredient: Lasagneplattor, Total Quantity: 200
  Ingredient: FetaOlja, Total Quantity: 75
  Ingredient: HönsStrimlor, Total Quantity: 450
  Ingredient: Spenat, Total Quantity: 100
  Ingredient: Vitlök, Total Quantity: 0
  Ingredient: Farfalle, Total Quantity: 500
  Ingredient: Potatismos, Total Quantity: 0
  Ingredient: Fiskpinnar, Total Quantity: 600
  Ingredient: RemouladeSås, Total Quantity: 0
  Ingredient: HönsStrimlorMarinad, Total Quantity: 500
  Ingredient: Matgrädde, Total Quantity: 8
  Ingredient: Paprika, Total Quantity: 1
  Ingredient: Ris, Total Quantity: 5
  Ingredient: Tomatpuree, Total Quantity: 60
  Ingredient: Persilja, Total Quantity: 0
  Ingredient: Parmesan